In [2]:
import json
import os
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"117718cf-b601-4a00-9164-3e4311468e45","PROXY_ID":"1b032d60-0671-498f-a117-6c2f355648fe","TENANT_URL":"https://tenant-jayasimha.dev.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"unskript","AWS_REGION":"us-west-2"}
os.environ["UNSKRIPT_REDIS_HOST"] = "redis-master.redis.svc.cluster.local"
os.environ["UNSKRIPT_TOKEN"] = "3BF5FBDA-0267-4961-9331-1AADF4BEB9B8"
os.environ["TENANT_URL"] = env["TENANT_URL"]
paramDict = {"channel": None, "elb_name": [], "region": []}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
channel = nbParamsObj.get('channel')
elb_name = nbParamsObj.get('elb_name')
region = nbParamsObj.get('region')

w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>To get AWS ELB unhealthy instances using unSkript actions.</strong></div>
</center><center>
<h2 id="Get-AWS-ELB-Unhealthy-Instances">Get AWS ELB Unhealthy Instances</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1. Get Unhealthy instances from ELB<br>2. Post Slack Message<code>
</code></p>

In [13]:
if elb_name and not region:
    raise SystemExit("Provide region for the ELB instances!")

<h3 id="Get-UnHealthy-EC2-Instances-for-Classic-ELB">Get UnHealthy EC2 Instances for Classic ELB</h3>
<p>In this action, we filter the instances from ELB which is unhealthy. This action only executes when the elb_name and region are given as parameters.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>elb_name</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable</strong>: <code>elb_instance</code></p>
</blockquote>

In [8]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_get_unhealthy_instances_printer(output):
    if output is None:
        return
    if output == []:
        print("All instances are healthy")
    else:
        pprint.pprint(output)


@beartype
def aws_get_unhealthy_instances(handle, elb_name: str, region: str) -> List:
    """aws_get_unhealthy_instances returns array of unhealthy instances

     :type handle: object
     :param handle: Object returned from task.validate(...).

     :type elb_name: string
     :param elb_name: Name of the ELB. Note: It ONLY supports Classic.

     :type region: string
     :param region: Name of the AWS Region.

     :rtype: Returns array of unhealthy instances
    """

    elbClient = handle.client('elb', region_name=region)
    res = elbClient.describe_instance_health(
        LoadBalancerName=elb_name,
    )

    unhealthy_instances = []
    for instance in res['InstanceStates']:
        if instance['State'] == "OutOfService":
            unhealthy_instances.append(instance)

    return unhealthy_instances


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "elb_name": "iter_item",
    "region": "region[0]"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "elb_name",
    "iter_parameter": "elb_name"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(elb_name)>0",
    "condition_result": true
    }''')
task.configure(outputName="elb_instance")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_unhealthy_instances, lego_printer=aws_get_unhealthy_instances_printer, hdl=hdl, args=args)

<h3 id="AWS-List-All-Regions">AWS List All Regions</h3>
<p>In this action, we list all the available regions from AWS if the user does not provide a region as a parameter.</p>
<blockquote>
<p>Output variable: <code>region</code></p>
</blockquote>

In [3]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Get-Unhealthy-instances-from-ELB">Get Unhealthy instances from ELB</h3>
<p>Here we will use unSkript&nbsp;<strong>Get Unhealthy instances from ELB</strong> action. This action is used to get all unhealthy instances from ELB, the instances which are out of service are considered unhealthy instances.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>elb_name</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable</strong>: <code>unhealthy_instances</code></p>
</blockquote>

In [4]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_get_unhealthy_ec2_instances_for_elb_printer(output):
    if output is None:
        return

    if isinstance(output, CheckOutput):
        pprint.pprint(output.json())
    else:
        pprint.pprint(output)


@beartype
def aws_get_unhealthy_ec2_instances_for_elb(handle, elb_name: str = "", region: str = "") -> CheckOutput:
    """aws_get_unHealthy_ec2_instances_for_elb gives unhealthy instances from ELB

        :type elb_name: string
        :param elb_name: Name of the elastic load balancer.

        :type region: string
        :param region: AWS region.

        :rtype: A tuple with execution results and a list of unhealthy instances from ELB
    """

    result = []
    all_regions = [region]
    elb_list = []
    if not region:
        all_regions = aws_list_all_regions(handle)

    if not elb_name:
        for reg in all_regions:
            try:
                asg_client = handle.client('elb', region_name=reg)
                response = aws_get_paginator(asg_client, "describe_load_balancers", "LoadBalancerDescriptions")
                for i in response:
                    elb_dict = {}
                    elb_dict["load_balancer_name"] = i["LoadBalancerName"]
                    elb_dict["region"] = reg
                    elb_list.append(elb_dict)
            except Exception as error:
                pass

    if elb_name and not region:
        for reg in all_regions:
            try:
                asg_client = handle.client('elb', region_name=reg)
                response = aws_get_paginator(asg_client, "describe_load_balancers", "LoadBalancerDescriptions")
                for i in response:
                    if elb_name in i["LoadBalancerName"]:
                        elb_dict = {}
                        elb_dict["load_balancer_name"] = i["LoadBalancerName"]
                        elb_dict["region"] = reg
                        elb_list.append(elb_dict)
            except Exception as error:
                pass

    if elb_name and region:
        try:
            elbClient = handle.client('elb', region_name=region)
            res = elbClient.describe_instance_health(LoadBalancerName=elb_name)
            for instance in res['InstanceStates']:
                data_dict = {}
                if instance['State'] == "OutOfService":
                    data_dict["instance_id"] = instance["InstanceId"]
                    data_dict["region"] = reg
                    data_dict["load_balancer_name"] = i["LoadBalancerName"]
                    result.append(data_dict)
        except Exception as e:
            pass

    for elb in elb_list:
        try:
            elbClient = handle.client('elb', region_name=elb["region"])
            res = elbClient.describe_instance_health(LoadBalancerName=elb["load_balancer_name"])
            for instance in res['InstanceStates']:
                data_dict = {}
                if instance['State'] == "OutOfService":
                    data_dict["instance_id"] = instance["InstanceId"]
                    data_dict["region"] = reg
                    data_dict["load_balancer_name"] = i["LoadBalancerName"]
                    result.append(data_dict)
        except Exception as e:
            pass

    if len(result) != 0:
        return CheckOutput(status=CheckOutputStatus.FAILED,
                   objects=result,
                   error=str(""))
    else:
        return CheckOutput(status=CheckOutputStatus.SUCCESS,
                   objects=result,
                   error=str(""))







task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not elb_name",
    "condition_result": true
    }''')
task.configure(outputName="unhealthy_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_unhealthy_ec2_instances_for_elb, lego_printer=aws_get_unhealthy_ec2_instances_for_elb_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output from step 1A and step 1B and return a list of dictionary items for the unhealthy instances from ELB.</p>
<blockquote>
<p><strong>Output variable:</strong> elb_instance_list</p>
</blockquote>

In [13]:
elb_instance_list = []
try:
    if elb_instance:
        for k, v in elb_instance.items():
            elb_dict = {}
            if len(v) > 0:
                for i in v:
                    elb_dict["instance_id"] = i
                    elb_dict["load_balancer_name"] = k
                    elb_dict["region"] = region[0]
                    elb_instance_list.append(elb_dict)
except Exception as e:
    try:
        if unhealthy_instances:
            for k, v in unhealthy_instances.items():
                if v.status == CheckOutputStatus.FAILED:
                    for instance in v.objects:
                        elb_instance_list.append(instance)
    except Exception as error:
        pass

<h3 id="Post-Slack-Message">Post Slack Message</h3>
<p>Here we will use unSkript <strong>Post Slack Message</strong> action. This action takes channel: str and message: str as input. This input is used to post the message to the slack channel.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>message</code>, <code>channel</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable</strong>: <code>message_status</code></p>
</blockquote>

In [14]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Unhealthy instances for elb:{elb_instance_list}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "channel",
    "condition_result": true
    }''')
task.configure(outputName="message_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS and slack legos to perform AWS action and this runbook fetches the unHealthy EC2 instances for Classic ELB and posts to a slack channel. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>